# PROJECT - COMP 6321 (Machine Learning)
# Somaye GHAHARI - Mehrnoosh AMJADI - Bikash
# 40106359 - -

This is a Jupyter Notebook.

This section imports necessary libraries.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import ast
import time
%matplotlib inline
sns.set_style('whitegrid')
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

Credit dataframe is loaded and process to convert to acceptable format.

In [2]:
df_credit = pd.read_csv("archive/credits.csv", dtype='unicode')

columns = ['cast', 'crew']

for column in columns:
    # use ast because json data has single quotes in the csv, which is invalid for a json object; it should be " normally
    df_credit[column] = df_credit[column].apply(lambda x: np.nan if pd.isnull(x) else ast.literal_eval(x))
df_credit["id"] = pd.to_numeric(df_credit["id"])
df_credit.head(100)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


credit datasave is in json format. 
Here some of its information like actors' and director's name and gender are seperated in different columns in df_credits_flattened dataframe.

In [3]:
start_time = time.time()
df_credits_flattened = pd.DataFrame(None,None,columns=['id',
                        'actor_1_gender','actor_2_gender','actor_3_gender',\
                        'actor_1_name','actor_2_name','actor_3_name',\
                        'director_gender','director_name'])
new = {'id':np.nan,\
              'actor_1_gender':np.nan,'actor_2_gender':np.nan,'actor_3_gender':np.nan,\
              'actor_1_name':np.nan,'actor_2_name':np.nan,'actor_3_name':np.nan,\
              'director_gender':np.nan,'director_name':np.nan}
for i,row in df_credit.iterrows():
    # dummy row
    newrow=new
    
    # fill id
    #newrow['id'] = int(row['id'])

    # fill cast
    n1=[obj['name'] for obj in row['cast'] ]
    n2=[obj['gender'] for obj in row['cast'] ]
    x=min( len(n1), 3)
    for c in range(x):
        newrow['actor_'+str(c+1)+'_name'] = n1[c]
        newrow['actor_'+str(c+1)+'_gender'] = int(n2[c])

    # fill crew    
    n1=[obj['name'] for obj in row['crew'] if obj["job"]=="Director"]
    n2=[obj['gender'] for obj in row['crew'] if obj["job"]=="Director"]
    if n1 and n2 :
        newrow['director_name']=n1[0]
        newrow['director_gender']=int(n2[0])
    df_credits_flattened = df_credits_flattened.append(newrow,ignore_index=True)
print("--- %s seconds ---" % (time.time() - start_time))


--- 1085.7177712917328 seconds ---


In [4]:
df_credits_flattened.replace('', np.nan, inplace=True) 
df_credits_flattened.replace('0', np.nan, inplace=True) 
df_credits_flattened.replace('[]', np.nan, inplace=True) 
df_credits_flattened.replace('0.0', np.nan, inplace=True) 

In [5]:
df_credits_flattened["director_gender"] = pd.to_numeric(df_credits_flattened["director_gender"])
df_credits_flattened["actor_1_gender"] = pd.to_numeric(df_credits_flattened["actor_1_gender"])
df_credits_flattened["actor_2_gender"] = pd.to_numeric(df_credits_flattened["actor_2_gender"])
df_credits_flattened["actor_3_gender"] = pd.to_numeric(df_credits_flattened["actor_3_gender"])
df_credits_flattened["id"] = df_credit["id"]
df_credits_flattened.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               45476 non-null  int64 
 1   actor_1_gender   45476 non-null  int64 
 2   actor_2_gender   45476 non-null  int64 
 3   actor_3_gender   45476 non-null  int64 
 4   actor_1_name     45476 non-null  object
 5   actor_2_name     45476 non-null  object
 6   actor_3_name     45476 non-null  object
 7   director_gender  45476 non-null  int64 
 8   director_name    45476 non-null  object
dtypes: int64(5), object(4)
memory usage: 2.4+ MB


Movie metadata dataframe is loaded into df_meta.

In [18]:
df_meta = pd.read_csv("archive/movies_metadata.csv")
df_meta.info()

c:\users\mehrnosh\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

this section drops unnecessary columns and convert them to numeric values.

In [19]:
df_meta.drop(['adult', 'belongs_to_collection', 'homepage', 'imdb_id', 'original_language', 'original_title',\
              'overview', 'poster_path', 'production_companies', 'production_countries', 'runtime',\
              'spoken_languages', 'status', 'tagline','video'], axis=1, inplace=True)

In [20]:
df_meta.replace('', np.nan, inplace=True) 
df_meta.replace('0', np.nan, inplace=True) 
df_meta.replace('[]', np.nan, inplace=True) 
df_meta.replace('0.0', np.nan, inplace=True) 
df_meta.replace(0.0, np.nan, inplace=True)

df_meta['budget'] = pd.to_numeric(df_meta['budget'], errors='coerce')

df_meta['id'] = pd.to_numeric(df_meta['id'], errors='coerce')

df_meta['popularity'] = pd.to_numeric(df_meta['popularity'], errors='coerce')


df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   budget        8890 non-null   float64
 1   genres        43024 non-null  object 
 2   id            45463 non-null  float64
 3   popularity    45394 non-null  float64
 4   release_date  45379 non-null  object 
 5   revenue       7408 non-null   float64
 6   title         45460 non-null  object 
 7   vote_average  42462 non-null  float64
 8   vote_count    42561 non-null  float64
dtypes: float64(6), object(3)
memory usage: 2.6+ MB


Two dataframe movie credits and movie metadata are merged here.

In [21]:
# merged_df = pd.concat([df_meta, credits_flattened], axis=1)
merged_df = pd.merge(df_meta, df_credits_flattened, on=['id'], how='inner')
rate = merged_df['revenue'] / merged_df['budget']
merged_df.insert(3, "rate(revenue/budget)", rate, True) 
merged_df.info()
merged_df.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45538 entries, 0 to 45537
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                8912 non-null   float64
 1   genres                43096 non-null  object 
 2   id                    45538 non-null  float64
 3   rate(revenue/budget)  5393 non-null   float64
 4   popularity            45469 non-null  float64
 5   release_date          45451 non-null  object 
 6   revenue               7428 non-null   float64
 7   title                 45535 non-null  object 
 8   vote_average          42531 non-null  float64
 9   vote_count            42630 non-null  float64
 10  actor_1_gender        45538 non-null  int64  
 11  actor_2_gender        45538 non-null  int64  
 12  actor_3_gender        45538 non-null  int64  
 13  actor_1_name          45538 non-null  object 
 14  actor_2_name          45538 non-null  object 
 15  actor_3_name       

,budget,genres,id,rate(revenue/budget),popularity,release_date,revenue,title,vote_average,vote_count,actor_1_gender,actor_2_gender,actor_3_gender,actor_1_name,actor_2_name,actor_3_name,director_gender,director_name
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,12.451801,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,2,2,2,Tom Hanks,Tim Allen,Don Rickles,2,John Lasseter
1,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,4.043035,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,2,2,1,Robin Williams,Jonathan Hyde,Kirsten Dunst,2,Joe Johnston
2,NaN,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,NaN,11.712900,1995-12-22,NaN,Grumpier Old Men,6.5,92.0,2,2,1,Walter Matthau,Jack Lemmon,Ann-Margret,2,Howard Deutch
3,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357.0,5.090760,3.859495,1995-12-22,81452156.0,Waiting to Exhale,6.1,34.0,1,1,1,Whitney Houston,Angela Bassett,Loretta Devine,2,Forest Whitaker
4,NaN,"[{'id': 35, 'name': 'Comedy'}]",11862.0,NaN,8.387519,1995-02-10,76578911.0,Father of the Bride Part II,5.7,173.0,2,1,2,Steve Martin,Diane Keaton,Martin Short,2,Charles Shyer


In the following sections, total revenue of each director is caculated and is added to dataframe as revenue_y(last column).

In [22]:
def create_comparison_database(name, value, x, no_films, df):
    
    comparison_df = df.groupby(name, as_index=False)
    
    if x == 'mean':
        comparison_df = comparison_df.mean()
    elif x == 'median':
        comparison_df = comparison_df.median()
    elif x == 'sum':
        comparison_df = comparison_df.sum() 
        
    name_count_key = df[name].value_counts().to_dict()
    comparison_df['films'] = comparison_df[name].map(name_count_key)
    return comparison_df[comparison_df['films'] >= no_films][[ name, value]]

In [23]:
x=create_comparison_database('director_name','revenue','sum', 0, merged_df)
final_df = pd.merge(merged_df, x, on=['director_name'], how='inner', suffixes=('','_director_sum'))
final_df.head(10)

,budget,genres,id,rate(revenue/budget),popularity,release_date,revenue,title,vote_average,vote_count,actor_1_gender,actor_2_gender,actor_3_gender,actor_1_name,actor_2_name,actor_3_name,director_gender,director_name,revenue_director_sum
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,12.451801,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,2,2,2,Tom Hanks,Tim Allen,Don Rickles,2,John Lasseter,2.256015e+09
1,120000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",9487.0,3.027157,16.869209,1998-11-25,363258859.0,A Bug's Life,6.8,2379.0,2,1,1,Kevin Spacey,Julia Louis-Dreyfus,Hayden Panettiere,2,John Lasseter,2.256015e+09
2,90000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",863.0,5.526299,17.547693,1999-10-30,497366869.0,Toy Story 2,7.3,3914.0,2,2,1,Tom Hanks,Tim Allen,Joan Cusack,2,John Lasseter,2.256015e+09
3,NaN,"[{'id': 16, 'name': 'Animation'}]",13925.0,NaN,7.160378,1986-08-17,NaN,Luxo Jr.,7.1,148.0,0,0,0,Néstor Cantillana,Juan Pablo Miranda,Antonella Ríos,2,John Lasseter,2.256015e+09
4,120000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",920.0,3.849860,18.907948,2006-06-08,461983149.0,Cars,6.6,3991.0,2,2,1,Owen Wilson,Paul Newman,Bonnie Hunt,2,John Lasseter,2.256015e+09
5,200000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",49013.0,2.799262,13.693002,2011-06-11,559852396.0,Cars 2,5.8,2088.0,2,2,2,Owen Wilson,Larry the Cable Guy,Michael Caine,2,John Lasseter,2.256015e+09
6,NaN,"[{'id': 16, 'name': 'Animation'}]",13927.0,NaN,6.058028,1988-08-01,NaN,Tin Toy,6.3,95.0,1,2,2,Jenna Fischer,Chris Messina,Topher Grace,2,John Lasseter,2.256015e+09
7,NaN,"[{'id': 16, 'name': 'Animation'}]",13926.0,NaN,3.907112,1987-08-17,NaN,Red's Dream,6.3,76.0,2,0,2,Paul Anderson,Calum MacNab,Daniel Mays,2,John Lasseter,2.256015e+09
8,NaN,"[{'id': 16, 'name': 'Animation'}]",13928.0,NaN,5.509405,1989-01-01,NaN,Knick Knack,7.1,135.0,1,1,2,Martina García,Icíar Bollaín,Yon González,2,John Lasseter,2.256015e+09
9,NaN,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",13934.0,NaN,7.111223,2006-07-27,NaN,Mater and the Ghostlight,6.2,96.0,2,2,0,Larry the Cable Guy,Owen Wilson,Michael Wallis,2,John Lasseter,2.256015e+09


In this section, median of popularity of first actor is caculated and is added to dataframe as popularity_y(last column).

In [24]:
x=create_comparison_database('actor_1_name','popularity','median', 0, final_df)
final_df = pd.merge(final_df, x, on=['actor_1_name'], how='inner', suffixes=('', '_actor1_median'))

final_df.head(10)

,budget,genres,id,rate(revenue/budget),popularity,release_date,revenue,title,vote_average,vote_count,actor_1_gender,actor_2_gender,actor_3_gender,actor_1_name,actor_2_name,actor_3_name,director_gender,director_name,revenue_director_sum,popularity_actor1_median
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,12.451801,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,2,2,2,Tom Hanks,Tim Allen,Don Rickles,2,John Lasseter,2.256015e+09,10.237996
1,90000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",863.0,5.526299,17.547693,1999-10-30,497366869.0,Toy Story 2,7.3,3914.0,2,2,1,Tom Hanks,Tim Allen,Joan Cusack,2,John Lasseter,2.256015e+09,10.237996
2,60000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",363676.0,3.974501,18.275689,2016-09-07,238470033.0,Sully,7.0,2106.0,2,2,1,Tom Hanks,Aaron Eckhart,Laura Linney,2,Clint Eastwood,3.100675e+09,10.237996
3,75000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",6538.0,1.586672,9.979162,2007-12-19,119000410.0,Charlie Wilson's War,6.5,342.0,2,2,1,Tom Hanks,Philip Seymour Hoffman,Julia Roberts,2,Mike Nichols,8.059136e+08,10.237996
4,52000000.0,"[{'id': 18, 'name': 'Drama'}]",568.0,6.831499,18.761467,1995-06-30,355237933.0,Apollo 13,7.3,1637.0,2,2,2,Tom Hanks,Bill Paxton,Kevin Bacon,2,Ron Howard,3.714152e+09,10.237996
5,8000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 14, 'nam...",2619.0,8.727667,8.441602,1984-03-09,69821334.0,Splash,6.1,328.0,2,1,2,Tom Hanks,Daryl Hannah,Eugene Levy,2,Ron Howard,3.714152e+09,10.237996
6,125000000.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",591.0,6.142564,18.566706,2006-05-17,767820459.0,The Da Vinci Code,6.5,2773.0,2,1,2,Tom Hanks,Audrey Tautou,Ian McKellen,2,Ron Howard,3.714152e+09,10.237996
7,150000000.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",13448.0,2.377423,22.211558,2009-05-13,356613439.0,Angels & Demons,6.5,2183.0,2,2,1,Tom Hanks,Ewan McGregor,Ayelet Zurer,2,Ron Howard,3.714152e+09,10.237996
8,75000000.0,"[{'id': 53, 'name': 'Thriller'}]",207932.0,2.933617,18.311054,2016-10-13,220021259.0,Inferno,5.7,2316.0,2,1,2,Tom Hanks,Felicity Jones,Omar Sy,2,Ron Howard,3.714152e+09,10.237996
9,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",29968.0,NaN,8.750517,1986-07-29,NaN,Nothing in Common,6.1,30.0,2,2,1,Tom Hanks,Jackie Gleason,Eva Marie Saint,2,Garry Marshall,1.633668e+09,10.237996
